# Lab 2: Cosmological Rates, Star Formation Histories, & Detectability
#### Mike Zevin, Monica Gallegos-Garcia

Goals:

* Learn how to calculate rate densities from a Transient Population
* Understand how star formation history is convolved with the Transient Population to determine rates
* Explore variations in star formation history and metallicity evolution assumptions
* Apply detectability criteria to get observable rate predictions for transient sources

___

In this lab, we will piggyback off of the `TransientPopulation` class that we created in the previous lab to generate observable predictions. We will start by calculating rate densities for a particular transient population. These rate densities depend on assumptions for the star formation history (SFH) and metallicity evolution of the universe. There are significant uncertainties in both the SFH and metallicity evolution, so next we'll explore how to vary these and compare predictions for different assumptions. Lastly, we will dive into detectability. Each transient source has its own set of selection effects that depend on the detector. This detectability will turn our cosmological rate density into an actual detection rate. 
___

## 1. Calculating Rate Densities from a Transient Population

Let's first read in the Transient Population that we created in the previous lab. 

In [ ]:
import numpy as np
import pandas as pd
from posydon.popsyn.synthetic_population import TransientPopulation

BBH_mergers = TransientPopulation(filename='BBH_mergers.h5', transient_name='BBH')

The `Rates` class is another class in POSYDON. It is constructed from a particular `TransientPopulation`, and needs two things: 
- model weights that determine the contribution of each system in the `TransientPopulation` (in $M_\odot^{-1}$)
- a star formation history model

The `Rates` class is a child class of the `TransientPopulation` class, which is in turn a child of the `Population` class, and therefore inherets information from these parent classes. 

We will first calculate cosmic weights associated with our transient population of BBH mergers. We specify the star formation history model parameters as a dictionary. For now we'll just use some basic parameter --- we'll get into variations in the star formation history model later in this lab. 

In [ ]:
SFH_model = {'SFR':'Madau+Fragos17',
            'sigma':0.39,
            'normalise':True,
            'Z_min':0,
            'Z_max':None}

BBH_merger_rates = BBH_mergers.calculate_cosmic_weights('MadauFragos2017', model_weights='full_IMF', MODEL_in=SFH_model)

These get saved as a new class instance with the same file name as the transient population (in our case, 'BBH_mergers.h5'). We can save multiple star formation history models with differing SFH identifiers. 

This new `rates` object we just created contains a few new things: 
- `z_birth`: the redshift and age of the universe at which we probe the star formation
- `z_events`: the redshift at which an event takes place (so, the redshift that corresponds to $t_\mathrm{birth} - t_\mathrm{delay}$ where $t_\mathrm{delay}$ is the delay time between stellar birth and the event, in our case BBH mergers); note that the values are NaN if the formation time + delay time is longer than the Hubble time
- `weights`: the weight of the event based on the SFR, its metallicity distribution and evolution, and its weight in the population

You can read in an instance of this class from our main file. Similar to before, we need to specifiy a transient name identifier (in our case, 'BBH') and a SFH identifier ('MadauFragos2017'). Once again, our main file can hold multiple different types of transients and SFH assumptions, and these instances are just called by specifying the associated `transient_name` and `SFH_identifier`. You can have as many transients and star formation histories in your population file as you want!

In [ ]:
from posydon.popsyn.synthetic_population import Rates

BBH_merger_rates = Rates(filename='BBH_mergers.h5', transient_name='BBH', SFH_identifier='MadauFragos2017')

You can directly work with the weights and the z_events to get events at specific redshifts. Let's take a second to look at the distribution of event masses at a few different redshifts: z=0, z=2, and z=10. 

<div class="alert alert-success">

## MINI-EXERCISE:

1. Write a piece of code that plots the primary mass distribution of BBHs from stars that were born at z=0.1, z=2, and z=10.
   
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint (click to reveal):</summary></b>

First get the index of the redshift bin from `BBH_merger_rates.z_birth`, then get the weights associated with this redshift index from `BBH_merger_rates.weights`, and lastly get the masses from `BBH_merger_rates.population`.
    
</details>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal):</summary></b>
    
```python
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

for idx, z in enumerate([0.1, 2, 10]):
    # first, find the redshift index closet to the one that we are looking for in z_birth
    z_sample_idx = np.argwhere(BBH_merger_rates.z_birth['z'] > z)[0][0]

    # next, find the weight associated with each systems that came from stars born at this redshift
    weights = BBH_merger_rates.weights[z_sample_idx].values

    # next, get the primary masses of the BHs from each system
    masses = BBH_merger_rates.population['S1_mass'].values

    # now, plot the weighted histogram
    _ = ax.hist(masses, weights=weights, bins=28, range=(5,60), histtype='step', density=True, alpha=0.5, label='z = {:0.1f}'.format(z))

plt.legend()
```
    
</details>

From the `Rates` class you created, one can easily calculate and plot the rate density as a function of redshift for your transient event of choice using internal POSYDON class methods. This can be done for the full transient population, or by individual subchannels if you set `channels=True`. The index of the resultant dataframe is the redshift at which the rate is calculated. 

In [ ]:
BBH_merger_rates.calculate_intrinsic_rate_density(channels=True)   # this can be called after it is calculated using rates.intrinsic_rate_density

Let's now plot the intrinsic rate evolution of your transient population with the SFH we assumed above using the `plot_intrinsic_rate` method!

In [ ]:
BBH_merger_rates.plot_intrinsic_rate(channels=True, xlim=(0,10), ylim=(1e-2,1e3))

Sometimes you might want some more details about the actual population and its properties. You can experimeent with this using the `plot_hist_properties` method of the `Rates` class.
___

## 2: SFH Assumptions

Similar to initial conditions via the `TransientPopulations` class's `model_weights`, there are various choices of star formation histories that can be applied with the `Rates` class's `calculate_cosmic_weights`. In this section of the lab, we will explore some of these variations to see how they impact our populations. 

In [ ]:
from IPython.display import Image
Image(filename='sfh_variations.png', width=750)

The figure above shows variations in the star formation history of the universe, either using parametric forms of the SFH based on observations (Madau+Dickinson14, Madau+Fragos17, Neijssel+19) or cosmological simulations (Nelson+18). 

In [ ]:
Image(filename='met_variations.png', width=750) 

The figure above shows variations in the metallicity distribution and evolution in the universe at different redshift slices. The dotted and dashed linestyles assuming a parametric metallicity evolution with log-normal dispersions of differing width, and the solid line shows the predictions from IllustrisTNG cosmological simulations. 

Both the star formation history and metallicity evolution constitute a major uncertainty in transient predictions from massive-star evolution, but are things that can be explored in post-processing (i.e., applied after the binaries have been simulated). Below we give parameters for various different star formation history and metallicity evolution models. 

In [ ]:
# Star Formation History models
SFH_models = {
    'IllustrisTNG' :{'SFR' : 'IllustrisTNG',
                        'normalise':True,
                        'Z_min':0, 
                        'Z_max':None},
    'MadauFragos_17' : {'SFR':'Madau+Fragos17',
                        'sigma':0.39,
                        'normalise':True,
                        'Z_min':0,
                        'Z_max':None},
    'Neijssel_19' : {'SFR':'Neijssel+19',
                        'sigma':0.39,
                        'normalise':True,
                        'Z_min':0,
                        'Z_max':None},
    'Fujimoto24': {'SFR':'Fujimoto+24',
                        'sigma':0.5,
                        'normalise':True,
                        'Z_min':0,
                        'Z_max':None},
}

Certain models have certain unique parameters. For example, the parametric models (e.g., `Neijssel+19`, `Madau+Fragos17`, `Fujimoto+24`) allow for one to specify the dispersion in the log-normal metallicity distribution with the `sigma` key. One can also set the minimum and maxmimum metallicity with `Z_min` and `Z_max`, respectively. The `normalise` key determines whether to renormalize the metallicity distribution so that it integrates to 1 at each redshift if `Z_min` or `Z_max` are specified. 

Let's compare the BBH merger rate for these differing metallicity distributions below

In [ ]:
# first, we calculate the intrinsic rates from each of the models
for sfh in SFH_models.keys():
    print(sfh)
    BBH_merger_rates = BBH_mergers.calculate_cosmic_weights(sfh, model_weights='full_IMF', MODEL_in=SFH_models[sfh])

Next, we'll plot the intrinsic rate density for merging BBHs in each of these models. 

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

for sfh in SFH_models.keys():
    BBH_merger_rates = Rates(filename='BBH_mergers.h5', transient_name='BBH', SFH_identifier=sfh)
    rates = BBH_merger_rates.calculate_intrinsic_rate_density()
    ax.plot(rates.index, rates.total, label=sfh)


ax.set_xlabel('redshift')
ax.set_ylabel('$R$ [Gpc$^{-3}$ yr$^{-1}$]')
ax.set_xlim(0,20)
ax.set_ylim(bottom=0)

plt.legend()

___
## 3: Detectability

So far, we've only been concerned with the _intrinsic_ rate density of transient events. However, the _detectable_ rate (how many events one would expect to detect per unit time) and _detecable_ population (intrinsic properties of the detectable population) depends on detector sensitivity. This could be, for example, a detection fraction of supernovae for a particular telescope survey, or the detection efficiency of gravitational-wave mergers by the ground-based gravitational wave detector network. These are what we call _selection effects_. 

Detection probability weights can be calculated for a transient population in a similar way to how transient populations are created: the `calculate_observable_population` function takes a `observable_func`, which describes the observability of a transient. 

Compact binaries have well-known selection effects. The figure below shows the detection probability as functions of mass, mass ratio, effective spin, and redshift. 

In [ ]:
from IPython.display import Image
Image(filename='GW_selection_function.png', width=1000)

Below we'll show how to define an observable population of BBH merger based on estimated sensitivity of a design-sensitivity network of current ground-based GW detectors. This is something where we have a pre-computed grid of detection probabilities based on the system's masses, mass ratio, effective spin, and redshift. But you can create your own function in something like the `DCO_wrapper` below that uses a different selection function than the `DCO_detectability`. We'll use a projected sensitivity for the LIGO-Virgo-KAGRA network operating in O4. 

In [ ]:
from posydon.popsyn.transient_select_funcs import DCO_detectability
BBH_merger_rates = Rates(filename='BBH_mergers.h5', transient_name='BBH', SFH_identifier='MadauFragos2017')

def DCO_wrapper(transient_chunk, z_events_chunk, weights_chunk):
    # once again, these can be done in chunks of the population, but we don't need to worry about it given the small population size we're using
    sensitivity = 'O4low_H1L1V1'
    return DCO_detectability(sensitivity, transient_chunk, z_events_chunk, weights_chunk, verbose=False)

BBH_merger_rates.calculate_observable_population(DCO_wrapper, 'O4low_H1L1V1')

# We can now access this observable population
BBH_merger_rates.observable_population('O4low_H1L1V1')

This table provides weights in units of yr$^{-1}$. If we wanted, for example, to calculate the detectable rate from the detector we specifed, we can sum up the weights for each system across all redshift columns. 

In [ ]:
detection_rate = BBH_merger_rates.observable_population('O4low_H1L1V1').sum().sum()
print('Number of BBH detections predicted from O4-seneitivity detectors: {:0.2f} per year'.format(detection_rate))

A little on the high end, but this is just with a default model. Varying parameters in the population ini file, the model weights, or assumptions about the star formation history, can bring this down. 

Selection effects are dependent on the properties of the system and where in the universe the transient event happened. For example, GW detectors are more sensitive to higher-mass systems (up to a certain point), with unequal-mass systems and systems with spins aligned opposite the orbital angular momentum having a more minor effect of decreasing the detectability. 

With POSYDON functionality, we can take a look at what the observable population is compared to the intrinsic population using `plot_hist_properties`. Let's look at how selection effects impact the observed distribution of BBH masses and spins. 

In [ ]:
fig, ax = plt.subplots(1,1)

BBH_merger_rates.plot_hist_properties('S1_mass', intrinsic=True, observable='O4low_H1L1V1', bins=np.linspace(0,60,31), normalise=True, ax=ax, label='S1', show=False)
ax.set_ylabel('Rate density [normalised]')   # Note: we normalized this since otherwise the observable population would be so small that we wouldn't see it in the plot!
ax.set_xlabel('Mass [Msun]')
ax.set_xlim(0,60)
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1)

BBH_merger_rates.plot_hist_properties('mass_ratio', intrinsic=True, observable='O4low_H1L1V1', bins=np.linspace(0,1,41), normalise=True, ax=ax, show=False)
ax.set_ylabel('Rate density [normalised]')
ax.set_xlabel('Mass ratio')
ax.set_xlim(0,1)
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1)

BBH_merger_rates.plot_hist_properties('chi_eff', intrinsic=True, observable='O4low_H1L1V1', bins=np.linspace(-1,1,41), normalise=True, ax=ax, show=False)
ax.set_ylabel('Rate density [normalised]')
ax.set_xlabel('Effective Inspiral Spin')
ax.set_xlim(-1,1)
ax.legend()
plt.show()

You can see that the observable population favors high masses, equal mass ratios, and positive effective spins compared to the underlying population. 
___

## 4: Exercise

<div class="alert alert-success">

## EXERCISE

So far, we've been focused on BBH mergers detected via gravitational waves. Let's try to recreate the rate density evolution and detectability of another transient: Type II supernovae. We'll be using everything learned in this lab and the previous lab to do this. 

1. Create a new `TransientPopulation` that targets successful core collapse (i.e., a NS is formed) of both the primary and the secondary star. We'll also want to make sure that the supernova type is a CCSN using `S1_SN_type` and `S2_SN_type` from the oneline. Note: you can create a `Population` with everything in these files since almost all go through core collapse. 
2. Calculate model weights for this transient population assuming the `full_IMF` that uses a Kroupa IMF from the previous lab. 
3. Plot the intrinsic rate density of Type II supernovae assuming a star formation history and metallicity evolution from the IllustrisTNG simulations.
4. **Hard**: Assume some simple and artificial selection function that is dependent on the luminosity distance of the SN (that scales as $(d_L / 1 Mpc)^{-2}$ and the mass of the progenitor (that scales as $(M / M_\odot)^{3}$. Note that you'll have to use astropy's `cosmo.luminosity_distance(z)` function to get luminosity distances from redshifts, and it's helpful to create an interpolant for this ahead of time to speed things up with scipy's `interp1d`. The detectability function takes in a transient population, z_events, and weights and returns a 2D array of new weights (the original weights times a detection probability, which ranges from 0 to 1). 
5. Plot the intrinsic and detectable mass distribution of the SN progenitors assuming these selection effects. 
   
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal):</summary></b>
    
```python
import numpy as np
import pandas as pd
from posydon.popsyn.synthetic_population import Population
from posydon.popsyn.synthetic_population import TransientPopulation
from posydon.popsyn.synthetic_population import Rates
from astropy.cosmology import Planck18 as cosmo
from scipy.interpolate import interp1d



# read in population files
print("Reading in population files...")
pop_files = ['1e-04_Zsun_population.h5',
             '1e-03_Zsun_population.h5',
             '1e-02_Zsun_population.h5',
             '1e-01_Zsun_population.h5',
             '2e-01_Zsun_population.h5',
             '4.5e-01_Zsun_population.h5',
             '1e+00_Zsun_population.h5',
             '2e+00_Zsun_population.h5']   

for file in pop_files:
    pop = Population(file)

    # let's also determine the formation channels for each system so we have them for later
    pop.calculate_formation_channels(mt_history=True)

    # export all systems, we'll parse the CCSN info when we create the transient population
    pop.export_selection(pop.indices, 'CoreCollapse.h5', append=True)



# create transient population
print("\n\n\nCreating transient population...")

CC_pop = Population('CoreCollapse.h5')
def CCSN_selection_function(history_chunk, oneline_chunk, formation_channel_chunk):
    '''Select succesfull CCSN events (NS formation) and return their time and metallicity.

    This function is used for transient population creation.
    '''

    # primary star
    S1CC_mask = oneline_chunk['S1_SN_type'] == 'CCSN'

    S1CC_hist = history_chunk.loc[oneline_chunk[S1CC_mask].index.tolist()]
    S1CC = S1CC_hist[(S1CC_hist['event'] == 'CC1').shift(1, fill_value=False)]
    successfull = S1CC[S1CC['S1_state'] == 'NS']

    CC1_dataframe = pd.DataFrame()
    CC1_dataframe['time'] = successfull['time'] / 1e6
    CC1_dataframe['metallicity'] = oneline_chunk.loc[successfull.index.tolist()]['metallicity']
    CC1_dataframe['mass'] = oneline_chunk.loc[successfull.index.tolist()]['S1_mass_i']


    # secondary star
    S2CC_mask = oneline_chunk['S2_SN_type'] == 'CCSN'
    S2CC_hist = history_chunk.loc[oneline_chunk[S2CC_mask].index.tolist()]
    S2CC = S2CC_hist[(S2CC_hist['event'] == 'CC2').shift(1, fill_value=False)]
    successfull = S2CC[S2CC['S2_state'] == 'NS']

    CC2_dataframe = pd.DataFrame()
    CC2_dataframe['time'] = successfull['time'] / 1e6
    CC2_dataframe['metallicity'] = oneline_chunk.loc[successfull.index.tolist()]['metallicity']
    CC2_dataframe['mass'] = oneline_chunk.loc[successfull.index.tolist()]['S2_mass_i']

    out_dataframe = pd.concat([CC1_dataframe, CC2_dataframe])

    return out_dataframe
    
CCSN = CC_pop.create_transient_population(CCSN_selection_function, 'CCSN')



# calculate model weights
print("\n\n\nCalculating model weights...")

full_IMF = {'number_of_binaries': 10000,
 'binary_fraction_scheme': 'const',
 'binary_fraction_const': 0.7,
 'star_formation': 'burst',
 'max_simulation_time': 13800000000.0,
 'primary_mass_scheme': 'Kroupa2001',
 'primary_mass_min': 0.01,
 'primary_mass_max': 200.0,
 'secondary_mass_scheme': 'flat_mass_ratio',
 'secondary_mass_min': 0.01*0.05,
 'secondary_mass_max': 200.0,
 'q_min':0,
 'q_max':1,
 'orbital_scheme': 'period',
 'orbital_period_scheme': 'Sana+12_period_extended',
 'orbital_period_min': 0.5,
 'orbital_period_max': 6000.0,
 'eccentricity_scheme': 'zero'}

_ = CCSN.calculate_model_weights('full_IMF', population_parameters=full_IMF)



# calculate rates
print("\n\n\nCalculating rates (this takes a little bit of time)...")
SFH_model = {'SFR' : 'IllustrisTNG',
             'normalise':True,
             'Z_min':0, 
             'Z_max':None}

CCSN_rates = CCSN.calculate_cosmic_weights('IllustrisTNG', model_weights='full_IMF', MODEL_in=SFH_model)   # this takes a little bit since there are so many systems



# plot merger rate density evolution
print("Plotting merger rate density evolution...")
CCSN_rates = Rates(filename='CoreCollapse.h5', transient_name='CCSN', SFH_identifier='IllustrisTNG')
CCSN_rates.calculate_intrinsic_rate_density()

CCSN_rates.plot_intrinsic_rate(channels=False, xlim=(0,13))



# calculate detectability
print("\n\n\nCalculating detectability...")

CCSN_rates = Rates(filename='CoreCollapse.h5', transient_name='CCSN', SFH_identifier='IllustrisTNG')
def ccsn_det(transient_pop_chunk, z_events_chunk, weights_chunk):

    z_grid = np.linspace(0,50,1000)
    dL_grid = cosmo.luminosity_distance(z_grid)
    z_to_dL_interp = interp1d(z_grid, dL_grid)   # Mpc

    dL = z_to_dL_interp(z_events_chunk)
    masses = np.ones_like(z_events_chunk) * np.atleast_2d(transient_pop_chunk['mass']).T

    pdet_weights = (dL/1.0)**(-2) * masses**(3.)
    always_detectable_mask = pdet_weights>1
    pdet_weights[always_detectable_mask] = 1.0

    return pdet_weights * weights_chunk

CCSN_rates.calculate_observable_population(ccsn_det, 'fake_telescope')

# We can now access this observable population
CCSN_rates.observable_population('fake_telescope')

detection_rate = CCSN_rates.observable_population('fake_telescope').sum().sum()
print('Number of CCSN detected by our fake telescope: {:0.2f} per year'.format(detection_rate))



# plot intrinsic versus observed pop
print("\n\n\nPlotting intrinsic versus observed population masses...")
fig, ax = plt.subplots(1,1)

CCSN_rates.plot_hist_properties('mass', intrinsic=True, observable='fake_telescope', bins=np.linspace(0,60,31), normalise=True, ax=ax, show=False)
ax.set_ylabel('Rate density [normalised]')   # Note: we normalized this since otherwise the observable population would be so small that we wouldn't see it in the plot!
ax.set_xlabel('Mass [Msun]')
ax.set_xlim(0,60)
ax.legend()
plt.show()
```
    
</details>